## this was notice to replace the download from the github make your own model dataset into huggingface


In [ ]:
!git clone https://huggingface.co/datasets/sameernotes/sakshi-handwriting

In [ ]:
%pip install opencv-python

In [ ]:
# title training
# Step 1: Import Libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
from google.colab import files  # preface for uploading files but recommended to use os methode to download files
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.font_manager as fm

In [ ]:
# Step 2: Verify GPU Availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.test.is_gpu_available())
print("Using T4 GPU:", tf.config.list_physical_devices('GPU'))

In [ ]:
# Step 3: Install a Devanagari Font
url = "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSansDevanagari/NotoSansDevanagari-Regular.ttf"
font_path = "[FONT_PATH]"  # Replace with your desired font path
r = requests.get(url)  # import requests
with open(font_path, "wb") as f:
    f.write(r.content)
font_path = 'C:/Coding/ocr_model/NotoSansDevanagari-Regular.ttf'   
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'Noto Sans Devanagari'

In [ ]:
# Step 4: Load Words and Images (No Upload Needed)
with open('/content/words.txt', 'r', encoding='utf-8') as f:
    words = f.read().splitlines()

image_dir = '/content/images'
images = []
for i in range(1, 36):  # 1.png to 35.png
    img_path = os.path.join(image_dir, f'{i}.png')
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 32))  # Resize to fixed size
    img = img / 255.0  # Normalize
    images.append(img)

images = np.array(images)
images = images[..., np.newaxis]  # Add channel dimension: (35, 128, 32, 1)

In [ ]:
# Step 5: Encode Labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(words)

# Step 6: Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

# Generate augmented data
augmented_images = []
augmented_labels = []
for i in range(len(images)):
    img = images[i]
    lbl = labels[i]
    img = img[np.newaxis, ...]  # Add batch dimension for datagen
    for _ in range(5):  # Generate 5 augmented versions per image
        aug_img = next(datagen.flow(img, batch_size=1))[0]
        augmented_images.append(aug_img)
        augmented_labels.append(lbl)

augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine original and augmented data
X = np.concatenate([images, augmented_images], axis=0)
y = np.concatenate([labels, augmented_labels], axis=0)

In [ ]:
# Step 7: Split Data (80% train, 20% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Build CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 32, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),  # Prevent overfitting
    layers.Dense(35, activation='softmax')  # 35 classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Step 9: Train the Model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Step 10: Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

# Step 11: Plot Training History
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Step 12: Save the Model (Using .keras format)
model.save('hindi_ocr_model.keras')
files.download('hindi_ocr_model.keras')  # Download to your local machine

# Step 13: Test on a Sample Image
sample_img = X_test[0][np.newaxis, ...]  # Take first test image
pred = model.predict(sample_img)
pred_label = label_encoder.inverse_transform([np.argmax(pred)])[0]
print(f"Predicted Word: {pred_label}")
plt.imshow(X_test[0].squeeze(), cmap='gray')
plt.title(f"Predicted: {pred_label}")
plt.show()

In [3]:
# @title save indexer
import pickle

with open('/content/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [ ]:
# @title test model

import cv2
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
!wget https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSansDevanagari/NotoSansDevanagari-Regular.ttf -O /usr/share/fonts/truetype/NotoSansDevanagari-Regular.ttf
font_path = '[FONT_PATH]'  # Replace with your desired font path
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = 'Noto Sans Devanagari'
model_path = '/content/hindi_ocr_model.keras'
model = tf.keras.models.load_model(model_path)
print("Model loaded successfully from", model_path)
encoder_path = '/content/label_encoder.pkl'
if not os.path.exists(encoder_path):
    raise FileNotFoundError(f"Label encoder {encoder_path} not found. Upload it or use numeric output.")
with open(encoder_path, 'rb') as f:
    label_encoder = pickle.load(f)
print("Label encoder loaded successfully from", encoder_path)
image_path = '/content/images/36.png'
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Image {image_path} not found. Ensure it’s in /content/images/")

img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (128, 32))  # Resize to match training input size
img = img / 255.0  # Normalize
img = img[np.newaxis, ..., np.newaxis]  # Add batch and channel dimensions: (1, 128, 32, 1)
pred = model.predict(img)
pred_label_idx = np.argmax(pred)
pred_label = label_encoder.inverse_transform([pred_label_idx])[0]
print(f"Predicted Word for 2.png: {pred_label}")
plt.imshow(img.squeeze(), cmap='gray')
plt.title(f"Predicted: {pred_label}")
plt.axis('off')
plt.show()